In [1]:
import sys
from collections import defaultdict
from itertools import combinations

import numpy as np

from src.dataset import X_TIME
from src.utils import deconvolve_precision, PLANES

PWD = '../..'
sys.path.append(PWD)

DATASET_PATH = '../../data/dataset/dataset_exp.pkl'
IMAGES_DIR = PWD + '/data/images/cfd_deconvolution/'

import pickle

from src.cfd import CFD

In [2]:
CFD_THRESHOLD = 0.2
CFD_N_BASELINE = 6

cfd = CFD(n_baseline=CFD_N_BASELINE, threshold=CFD_THRESHOLD)

with open(DATASET_PATH, 'rb') as file:
    dataset = pickle.load(file)

# Compute CFD timestamps

In [3]:
cfd_timestamps = defaultdict(list)

for plane_channel, data in dataset.items():
    if plane_channel == 't_cfd_avg':
        continue

    plane, channel = plane_channel
    X = data[0]

    t_cfd = cfd.predict(X_TIME, X)
    cfd_timestamps[(plane, channel)] = t_cfd

# Find mutual precisions

In [4]:
mutual_precisions = {}

for ch1, ch2 in combinations(cfd_timestamps.keys(), 2):
    if ch1[1] == ch2[1]:  # Only for corresponding channels
        ch1_timestamps, ch2_timestamps = cfd_timestamps[ch1], cfd_timestamps[ch2]
        differences = [abs(ch1_t - ch2_t) for ch1_t, ch2_t in zip(ch1_timestamps, ch2_timestamps) if
                       ch1_t is not None and ch2_t is not None]

        mutual_precisions[(ch1, ch2)] = np.std(differences)

In [5]:
channel_mutual_precisions = defaultdict(dict)
for ((x_p, x_ch), (y_p, y_ch)), precision in mutual_precisions.items():
    assert x_ch == y_ch
    channel_mutual_precisions[x_ch][(x_p, y_p)] = precision

In [6]:
channel_mutual_precisions

defaultdict(dict,
            {2: {(1, 2): 0.07040789491689162,
              (1, 3): 0.07459640560597511,
              (2, 3): 0.0856409201840996},
             11: {(1, 2): 0.07755414161223897,
              (1, 3): 0.10584888615222335,
              (2, 3): 0.1168213568520453}})

In [7]:
for ch, data in channel_mutual_precisions.items():
    for (p_1, p_2), prec in data.items():
        print(f'ch {ch:2}: (p{p_1} vs p{p_2}): {prec:0.3f}')

ch  2: (p1 vs p2): 0.070
ch  2: (p1 vs p3): 0.075
ch  2: (p2 vs p3): 0.086
ch 11: (p1 vs p2): 0.078
ch 11: (p1 vs p3): 0.106
ch 11: (p2 vs p3): 0.117


# Deconvolve

In [8]:
deconvolved_precisions = {}
for ch, prec_dict in channel_mutual_precisions.items():
    for p in PLANES:
        deconvolved_precisions[(p, ch)] = deconvolve_precision(p, prec_dict)

In [9]:
for (p, ch), prec in deconvolved_precisions.items():
    print(f'(p {p}, ch {ch:2}): {prec:0.3f}')

(p 1, ch  2): 0.056
(p 2, ch  2): 0.082
(p 3, ch  2): 0.089
(p 1, ch 11): 0.060
(p 2, ch 11): 0.092
(p 3, ch 11): 0.137
